# Crowd Detection Project - Image Exporter

This process generates training and testing data for image classification from video feeds given ground-truth data found in a feature class.

## Pseudocode

1. Import needed modules

2. Establish runtime variables
    - GIS
    - video root directory
    - labels feature class
    - labels FC camera field
    - labels FC label field
    - labels FC datetime field
    
3. Create Helper Functions:
    - Input: label FC record (datetime, label, camera) | Output: video url and timestamp

4. Iteration: For each record of the labeled feature class...
    - Run helper function to determine video url and timestamp
    - Determine output path
    - Run export using moviepy

### 1. Import needed modules

In [1]:
import arcgis
import moviepy.editor as mpy
import datetime
import os

### 2. Establish runtime variables

- GIS
- video root directory
- labels feature class
- labels FC camera field
- labels FC label field
- labels FC datetime field

In [2]:
gis = arcgis.gis.GIS("https://esrifederal.maps.arcgis.com", username="Anieto_esrifederal")

Enter password: ········


In [3]:
# Make reference to directories and other needed paths
video_root_dir = r"D:\5_Data\Transportation\Cameras\SunTrustPark"
labels_fc = r"D:\3_Sandbox_Projects\1806_CobbCounty_PedestrianDetection\Inputs\S123_Labels.gdb\PedestrianCount"
labels_fc_camera_field = "CameraID"
labels_fc_label_field = "degreesaturation"
labels_fc_datetime_field = "PhotoDateTime"

In [4]:
labels_sdf = arcgis.features.SpatialDataFrame.from_featureclass(labels_fc)

### 3. Create Helper Functions

In [5]:
# Input: List of video files with names HHhMMmSSs.avi, datetime object | Output: Video containing the datetime
def get_videopath_and_video_timestamp_from_label_timestamp(camera_dir, datetime_object, debug=False):
    
    # Get list of video files in the camera directory
    video_files = os.listdir(camera_dir)
    
    if debug:
        print("<<<< Debug >>>> ")
        print("\nInputs: ")
        print("\tObservation Date: {0}".format(datetime_object.strftime("%y%m%d")))
        print("\tObservation Time: {0}".format(datetime_object.strftime("%H:%M:%S")))
    
    # For each file in video files...
    for ix, video_file in enumerate(video_files):
        
        video_found = False
    
        # we need a datetime range...
        yymmdd = datetime_object.strftime("%y%m%d")
        start = datetime.datetime.strptime(video_files[ix]+" "+yymmdd, "%Hh%Mm%Ss.avi %y%m%d")
        clip = mpy.VideoFileClip(camera_dir + "\\" + video_files[ix])
        runtime = clip.duration
        clip.reader.close()
        end = start + datetime.timedelta(0, runtime)
        
        in_time_range = start <= datetime_object <= end
        
        if debug:
            print("\nRecording: {0}".format(video_file))
            print("\tCamera Dir: {0}".format(camera_dir))
            print("\tStart: {0}".format(start))
            print("\tRuntime: {0}".format(runtime))
            print("\tEnd: {0}".format(end))
            print("\tIn Range?: {0}".format(in_time_range))
        
        # so we can determine if the current timeslot exists there
        if in_time_range:
            
            # And get the video timestamp
            delta_video_timestamp = datetime_object - start
            
            return video_file, str(delta_video_timestamp)
        
    
    print("\nCould not find the needed timeslot in the video...")
            
    return None, None

In [6]:
# Input: label FC record (datetime, label, camera) | Output: video url and timestamp
def retrieve_video_and_time(camera, 
                            label, 
                            datetime_obj,
                            video_root_dir,
                            debug=False):
    
    # This function relies on a video root directory structure as follows:
    
    # Root_Path
        # yymmdd
            # Camera ID
                # HHhMMmSSs.avi
    
    # Parse out datetime object into needed formats
    yymmdd = datetime_obj.strftime("%y%m%d")
    hh = datetime_obj.strftime("%H")
    mm = datetime_obj.strftime("%M")
    ss = datetime_obj.strftime("%S")
    
    # Determine which video contains the needed time and at which time the label was recorded in the video
    camera_dir = "{0}\\{1}\\{2}".format(video_root_dir, yymmdd, camera)
    video_name, video_time = get_videopath_and_video_timestamp_from_label_timestamp(camera_dir, datetime_obj, debug)
    
    return "{0}\\{1}".format(camera_dir, video_name), video_time

### 4.Iteration: For each record of the labeled feature class...
    - Run helper function to determine video url and timestamp
    - Determine output path
    - Run export using moviepy

In [8]:
def main(label_fc, 
         labels_fc_camera_field, 
         labels_fc_label_field, 
         labels_fc_datetime_field, 
         video_root_dir, 
         outputs_dir, 
         debug=False):
    # Convert the labels fc into a spatial dataframe in python memory
    labels_sdf = arcgis.features.SpatialDataFrame.from_featureclass(labels_fc)
    total_observations = labels_sdf.shape[0]
    # Iterate on each record to start the image export
    for index, row in labels_sdf.iterrows():
        
        camera = row[labels_fc_camera_field] 
        label = row[labels_fc_label_field]
        pandas_timestamp = row[labels_fc_datetime_field]
        
        print("\n<<<< Exporting {0} of {1} >>>>".format(index, total_observations))
        print(camera)
        print(label)
        print(pandas_timestamp)
        print("\n")        
        
        # Convert the pandas timestamp to datetime object
        pdt = pandas_timestamp.to_pydatetime()
        # Correct for input data timezone difference
        pdt = pdt - datetime.timedelta(hours=4)
        print(pdt)

        # Parse out datetime object into needed formats
        yymmdd = pdt.strftime("%y%m%d")
        hh = pdt.strftime("%H")
        mm = pdt.strftime("%M")
        ss = pdt.strftime("%S")
            
        # Get the path to the right video and the timestamp in the video for the recorded label
        video_path, video_time =  retrieve_video_and_time(camera, label, pdt, video_root_dir, debug)
        if debug:
            print(video_path, video_time)
        
        if video_path and video_time:
            clip = mpy.VideoFileClip(video_path)
            output_label_dir = outputs_dir + "\\" + label
            
            if not os.path.exists(output_label_dir):
                os.makedirs(output_label_dir)
            
            clip.save_frame("{6}\\{0}_{1}_{2}_{3}{4}{5}.png".format(label, camera, yymmdd, hh, mm, ss, output_label_dir), t=video_time)
            clip.reader.close()

In [9]:
# Make reference to directories and other needed paths
video_root_dir = r"D:\5_Data\Transportation\Cameras\SunTrustPark"
labels_fc = r"D:\3_Sandbox_Projects\1806_CobbCounty_PedestrianDetection\Inputs\S123_Labels.gdb\PedestrianCount"
labels_fc_camera_field = "CameraID"
labels_fc_label_field = "degreesaturation"
labels_fc_datetime_field = "PhotoDateTime"
outputs_dir = r"D:\3_Sandbox_Projects\1806_CobbCounty_PedestrianDetection\Work\1_image-exporter\model_inputs"

# Run main
main(labels_fc, labels_fc_camera_field, labels_fc_label_field, labels_fc_datetime_field, video_root_dir, outputs_dir)


<<<< Exporting 0 of 323 >>>>
129
Medium
2018-06-13 01:58:00


2018-06-12 21:58:00

<<<< Exporting 1 of 323 >>>>
130
Low
2018-06-13 01:59:00


2018-06-12 21:59:00

<<<< Exporting 2 of 323 >>>>
129
Medium
2018-06-13 02:00:00


2018-06-12 22:00:00

<<<< Exporting 3 of 323 >>>>
129
Medium
2018-06-13 02:01:00


2018-06-12 22:01:00

<<<< Exporting 4 of 323 >>>>
130
Low
2018-06-13 02:01:00


2018-06-12 22:01:00

<<<< Exporting 5 of 323 >>>>
129
Medium
2018-06-13 02:02:00


2018-06-12 22:02:00

<<<< Exporting 6 of 323 >>>>
129
Medium
2018-06-13 02:04:00


2018-06-12 22:04:00

<<<< Exporting 7 of 323 >>>>
128
Low
2018-06-13 02:04:00


2018-06-12 22:04:00

Could not find the needed timeslot in the video...

<<<< Exporting 8 of 323 >>>>
128
None
2018-06-13 02:03:00


2018-06-12 22:03:00

Could not find the needed timeslot in the video...

<<<< Exporting 9 of 323 >>>>
128
Low
2018-06-13 02:02:00


2018-06-12 22:02:00

Could not find the needed timeslot in the video...

<<<< Exporting 10 of 323 >>

<<<< Exporting 90 of 323 >>>>
130
High
2018-06-13 02:30:00


2018-06-12 22:30:00

<<<< Exporting 91 of 323 >>>>
130
High
2018-06-13 02:30:00


2018-06-12 22:30:00

<<<< Exporting 92 of 323 >>>>
130
Medium
2018-06-13 02:28:00


2018-06-12 22:28:00

Could not find the needed timeslot in the video...

<<<< Exporting 93 of 323 >>>>
130
High
2018-06-13 02:27:00


2018-06-12 22:27:00

Could not find the needed timeslot in the video...

<<<< Exporting 94 of 323 >>>>
130
High
2018-06-13 02:27:00


2018-06-12 22:27:00

Could not find the needed timeslot in the video...

<<<< Exporting 95 of 323 >>>>
130
Medium
2018-06-13 02:26:00.000001


2018-06-12 22:26:00.000001

Could not find the needed timeslot in the video...

<<<< Exporting 96 of 323 >>>>
130
High
2018-06-13 02:23:00.000001


2018-06-12 22:23:00.000001

<<<< Exporting 97 of 323 >>>>
130
High
2018-06-13 02:23:00.000001


2018-06-12 22:23:00.000001

<<<< Exporting 98 of 323 >>>>
130
High
2018-06-13 02:22:00


2018-06-12 22:22:00

<<<< Exp

2018-06-14 21:49:00

<<<< Exporting 182 of 323 >>>>
129
High
2018-06-15 02:16:00


2018-06-14 22:16:00

Could not find the needed timeslot in the video...

<<<< Exporting 183 of 323 >>>>
129
Medium
2018-06-15 02:14:00


2018-06-14 22:14:00

Could not find the needed timeslot in the video...

<<<< Exporting 184 of 323 >>>>
130
Medium
2018-06-15 02:14:00


2018-06-14 22:14:00

<<<< Exporting 185 of 323 >>>>
129
High
2018-06-15 02:13:00


2018-06-14 22:13:00

Could not find the needed timeslot in the video...

<<<< Exporting 186 of 323 >>>>
130
Low
2018-06-15 01:48:00


2018-06-14 21:48:00

<<<< Exporting 187 of 323 >>>>
130
Medium
2018-06-15 02:14:00


2018-06-14 22:14:00

<<<< Exporting 188 of 323 >>>>
129
High
2018-06-15 02:13:00


2018-06-14 22:13:00

Could not find the needed timeslot in the video...

<<<< Exporting 189 of 323 >>>>
130
Low
2018-06-15 02:14:00


2018-06-14 22:14:00

<<<< Exporting 190 of 323 >>>>
130
Low
2018-06-15 01:46:00


2018-06-14 21:46:00

<<<< Exporting 191 of

2018-06-14 22:12:00

<<<< Exporting 255 of 323 >>>>
128
High
2018-06-15 02:11:00


2018-06-14 22:11:00

<<<< Exporting 256 of 323 >>>>
128
High
2018-06-15 02:10:00.000001


2018-06-14 22:10:00.000001

<<<< Exporting 257 of 323 >>>>
128
High
2018-06-15 02:10:00.000001


2018-06-14 22:10:00.000001

<<<< Exporting 258 of 323 >>>>
128
High
2018-06-15 02:09:00


2018-06-14 22:09:00

<<<< Exporting 259 of 323 >>>>
128
High
2018-06-15 02:09:00


2018-06-14 22:09:00

<<<< Exporting 260 of 323 >>>>
128
High
2018-06-15 02:09:00


2018-06-14 22:09:00

<<<< Exporting 261 of 323 >>>>
128
High
2018-06-15 02:08:00


2018-06-14 22:08:00

<<<< Exporting 262 of 323 >>>>
128
High
2018-06-15 02:08:00


2018-06-14 22:08:00

<<<< Exporting 263 of 323 >>>>
128
High
2018-06-15 02:08:00


2018-06-14 22:08:00

<<<< Exporting 264 of 323 >>>>
128
High
2018-06-15 02:07:00


2018-06-14 22:07:00

<<<< Exporting 265 of 323 >>>>
128
High
2018-06-15 02:06:00


2018-06-14 22:06:00

<<<< Exporting 266 of 323 >>>>
128
Hig

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\5_Data\\Transportation\\Cameras\\SunTrustPark\\180612\\None'

### Appendix A: UnitTests

In [ ]:
ix = 0

In [ ]:
ts = labels_sdf.iloc[ix]['PhotoDateTime']
ts

In [ ]:
camera = labels_sdf.iloc[ix]['CameraID']
camera

In [ ]:
pdt = ts.to_pydatetime()
pdt

In [ ]:
pdt = pdt - datetime.timedelta(hours=4)
pdt

In [ ]:
yymmdd = pdt.strftime("%y%m%d")
yymmdd

In [ ]:
hh = pdt.strftime("%H")
hh

In [ ]:
mm = pdt.strftime("%M")
mm

In [ ]:
ss = pdt.strftime("%S")
ss

In [ ]:
video_files = os.listdir("{0}\\{1}\\{2}".format(video_root_dir, yymmdd, camera))
video_files

In [ ]:
# Get the length of the video from metadata
file = "{0}\\{1}".format(r"D:\5_Data\Transportation\Cameras\SunTrustPark\180612\129", video_files[0])
file

In [ ]:
clip = mpy.VideoFileClip(file)
clip.duration

In [ ]:
date_obj = datetime.datetime(2018, 6, 14, 21, 44)
get_videopath_and_video_timestamp_from_label_timestamp(r"D:\5_Data\Transportation\Cameras\SunTrustPark\180614\128", date_obj, debug=True)